In [ ]:
#addprocs()
ENV["WINSTON_OUTPUT"] = :gtk
push!(LOAD_PATH, "../src/")
using SortSpikes, DistributedArrays
#using Gtk.ShortNames, Winston

In [ ]:
#make data structure for spike sorting
st=Array(Sorting,64)

for i=1:64
    st[i]=Sorting(DetectPower(),ClusterOSort(),AlignMax(),FeatureDD(),ReductionNone())
end

s=distribute(st);

In [ ]:
@sync @parallel for i=1:64
    s[i].rawSignal[:]=convert(Array{Int64,1},rand(Uint8,60000))
    s[i].rawSignal[100]=1000
end;

In [ ]:
@time firstrun_par(s)

In [ ]:
@time cal_par(s)

In [ ]:
@time onlinesort_par(s)

In [ ]:
#Make GUI
win=@Window(title="myGui")
g=@Grid()
push!(win,g)
c=[@Canvas(200,200) for i=1:32]
for i=1:4
    for j=1:8
        g[j,i]=c[sub2ind((8,4),j,i)]
    end 
end
showall(win)

#Display Winston Frame Plots on Canvases with 
f=Array(Any,32)
cu=Curve[Curve(1:50,collect(linspace(1,10000,50)), color="red") for i=1:32]

p=FramedPlot[FramedPlot() for i=1:32]

for i=1:32
    f[i]=Winston.Figure(c[i],p[i])
    Winston.addfig(Winston._display,i,f[i])
    add(p[i],cu[i])
    Winston.display(c[i],p[i])
end

In [ ]:
#get context for plots, and make clearing function
a=[get_context(c[i],p[i]) for i=1:32]
wipe=[wipeout(a[i],50,10000) for i=1:32]

#How many spikes are on the plot
count=zeros(Int,32)

for i=1:32
    Winston.paint(wipe[i],a[i].paintc)
    reveal(c[i],false)
end

In [ ]:
#callbacks to get position of mouse clicks

global xpos=zeros(Float64,32,2)
global ypos=zeros(Float64,32,2)
global mansort=falses(32)
global ac=a[1]

for i=1:32
    
    c[i].mouse.button1press = (w,event) -> begin 
        global xpos
        global ypos
        global ac
        xpos[i,1]=(event.x - ac.geom.x.a)/ac.geom.x.b
        ypos[i,1]=(ac.draw.ctx.surface.height - event.y - ac.geom.y.a)/ac.geom.y.b
    end
    c[i].mouse.button1release = (w,event) -> begin 
        global xpos
        global ypos
        global ac
        xpos[i,2]=(event.x - ac.geom.x.a)/ac.geom.x.b
        ypos[i,2]=(ac.draw.ctx.surface.height - event.y - ac.geom.y.a)/ac.geom.y.b
    end
    c[i].mouse.button3press = (w,event) -> begin 
        global mansort
        mansort[i] = !mansort[i]
    end
end

In [ ]:
waves=[convert(SharedArray,zeros(Int64,length(s[1].waveforms[1,1]))) for i=1:32 ,j=1:10];
#make the electrodes you want to be on the screen also point to waves
n=2
for i=1:32
    if i>s.cuts[1][n]
        n+=1
    end
    for j=1:10
        @spawnat n s[i].waveforms[j]=waves[i,j] #this might not point appropriately
    end
end

In [ ]:
@time for i=1:32
    #If too many elements on plot, wipe it to start
    if count[i]>20
        Winston.paint(wipe[i],a[i].paintc)
        count[i]=0
    end
    
    j=2
    #Loop over new spikes
    while waves[i,j][1]>0
        
        #get spike
        cu[i].y[:]=waves[i,j]
        
        #change color based on neuron cluster (assigned in first dummy wave)
        cu[1].attr.vals[1][:linecolor]=SortSpikes.colordict[waves[i,j][j]]
        
        #Plot spike
        plotline(i,c,cu,a)
        
        #keep track of number of spikes on graph
        count[i]+=1
        j+=1
    end
    
    #show graph after everything is plotted.
    reveal(c[i],false)
end